# transfer Learning first test

In [1]:
import tensorflow as tf
import os
import numpy as np
import cv2
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Input
from tensorflow.keras import losses
import matplotlib.pyplot as plt
from tensorflow.keras.applications import vgg16
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model

In [2]:
list_images = []
labels = []
for image_name in os.listdir("224_sized/"):
    if image_name.endswith('.jpg'):
        image = cv2.imread(f'224_sized/{image_name}')
        if image.shape != (224,224,3):
            print(image_name)
            
        label_name = image_name.replace('.jpg','.txt')
        with open(f'224_sized/{label_name}') as txt_file:
            txt = txt_file.read()
            
        label = int(txt.split()[0])
        labels.append(label)   
        list_images.append(image)

In [3]:
new_labels = np.zeros((len(labels),4))
print(new_labels.shape)
for i in range(len(labels)):
    new_labels[i,labels[i]] += 1
    
labels = new_labels

(414, 4)


In [4]:
list_images = np.array(list_images)
list_images = list_images.reshape(-1,224,224,3)

X_train, X_test, Y_train, Y_test = train_test_split(list_images,labels,test_size=.2,)
X_train, X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=.2)

In [5]:
def featur_extractor_layer(inputs):
    transfer_layer = vgg16.VGG16(include_top=0,input_shape=(224,224,3))(inputs)
    return transfer_layer

In [11]:
def base_model(inputs):
    x = featur_extractor_layer(inputs)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(4, activation='relu')(x)
    return x

In [12]:
def final_model(inputs):
    outputs = base_model(inputs)
    my_model = Model(inputs, outputs)
    return my_model

In [13]:
inputs = Input((224,224,3))
the_model = final_model(inputs)

In [14]:
categorical_loss = losses.categorical_crossentropy
the_model.compile(loss=categorical_loss)

In [ ]:
history = the_model.fit(X_train,Y_train,epochs=20,validation_data=(X_val,Y_val))

Epoch 1/20
